<h2>程序性能检测及优化</h2>

<h3>目标</h3>

在图像处理中你每秒需要做大量的运算，所以你的程序不仅要能给出正确的结果，同时还必须要快。所以这节我们将要学习：
<ul>
    <li>检测程序的效率</li>
    <li>一些能够提高程序效率的技巧</li>
    <li>你将要学到的函数有：cv2.getTickCount,cv2.getTickFrequency 等</li>
    </ul>
除了 OpenCV，Python 也提供了一个叫 time 的的模块。你可以用它来测量程序的运行时间 。另外一个叫做 profile 的模块会帮你得到一份关于你的程序 的详细报告。其中包含了代码中每个函数运行需要的时间以及每个函数被调用的次数。如果你正在使用 IPython 的话，所有这些特点都以一种用户友好的方式整合在一起了。我们会学习几个重要的。要想学到更加详细的知识就打开更多资源中的链接吧。

<h3>8.1 使用OpenCV 检测程序效率</h3>

cv2.getTickCount 函数返回从参考点到这个函数被执行的时钟数。所以当你在一个函数执行前后都调用它的话， 你就会得到这个函数的执行时 
（ 时钟数） 。<br>
cv2.getTickFrequency  返回时钟频率，或者说每秒钟的时钟数。所以
你可以按照下面的方式得到一个函数运行多少秒 

In [ ]:
e1 = cv2.getTickCount()
# your code execution
e2 = cv2.getTickCount()
time = (e2 - e1)/ cv2.getTickFrequency()

我们将会用下面的例子演示。下面的例子是用窗口大小不同（5，7，9）的核函数来做中值滤波 ：

In [ ]:


img1 = cv2.imread('messi5.jpg')

e1 = cv2.getTickCount()
for i in xrange(5,49,2):
    img1 = cv2.medianBlur(img1,i)
e2 = cv2.getTickCount()
t = (e2 - e1)/cv2.getTickFrequency()
print t

# Result I got is 0.521107655 seconds



<h4>注意</h4> 你也可以用 time 模块实现上面的功能。但是，用的函数是 time.time() 而不是 cv2.getTickCount。比较一下，两个结果的差别 吧。

<h3>8.2 OpenCv 中的默认优化</h3>

OpenCV 中的很多函数都被优化过（使用 SSE2，AVX 等） 。也包含一些没有被优化的代码。因此，如果我们的系统支持这些特性，我们应该利用它们(几乎所有现代处理器都支持它们)。它在编译时默认启用。所以OpenCV如果启用了优化的代码，就会运行未优化的代码。你可以使用函数 cv2.useOptimized() 来查看优化是否被开启了，使用函数 cv2.setUseOptimized() 来开启优化。让我们来看一个简单的例子吧。

In [ ]:
# check if optimization is enabled
In [5]: cv2.useOptimized()
Out[5]: True

In [6]: %timeit res = cv2.maedianBlur(img,49)
10 loops, best of 3: 34.9 ms per loop

# Disable it
In [7]: cv2.setUseOptimized(False)

In [8]: cv2.useOptimized()
Out[8]: False

In [9]: %timeit res = cv2.medianBlur(img,49)
10 loops, best of 3: 64.1 ms per loop



可见，优化的中值滤波比未优化的版本快2倍。如果您检查它的源代码，您可以看到中值筛选是SIMD优化的。因此，您可以使用此方法在代码的顶部启用优化(请记住它是默认启用的)。

<h3>8.3 在IPython 中检测程序效率</h3>

有时您可能需要比较两个类似操作的性能。IPython给您一个神奇的命令%timeit来执行此操作。它运行代码几次以获得更准确的结果。它们同样适用于测量单行代码。<br>


例如，你知道下面哪个加法操作更好，x = 5; y = x**2, x = 5; y = x*x, x = np.uint8([5]); y = x*x还是y = np.square(x) ?我们将在IPython shell中找到它的%timeit。

In [ ]:


In [10]: x = 5

In [11]: %timeit y=x**2
10000000 loops, best of 3: 73 ns per loop

In [12]: %timeit y=x*x
10000000 loops, best of 3: 58.3 ns per loop

In [15]: z = np.uint8([5])

In [17]: %timeit y=z*z
1000000 loops, best of 3: 1.25 us per loop

In [19]: %timeit y=np.square(z)
1000000 loops, best of 3: 1.16 us per loop



你可以看到，x = 5;y = x*x是最快的，比Numpy快20倍。如果您考虑数组的创建，它可能会更快地达到100倍。很酷,对吧?(Numpy devs正在研究这个问题)

<h4>注意：</h4>
Python标量操作比Numpy标量操作快。对于包含一两个元素的操作，Python标量比Numpy数组好。当数组的大小稍微大一点时，Numpy就占了优势。

我们再试一个例子。这一次，我们将比较cv2.countNonZero()和np.count_nonzero()的性能。

In [ ]:
In [35]: %timeit z = cv2.countNonZero(img)
100000 loops, best of 3: 15.8 us per loop

In [36]: %timeit z = np.count_nonzero(img)
1000 loops, best of 3: 370 us per loop

看，OpenCV函数比Numpy函数快25倍。

<h4>注意：</h4>
通常，OpenCV函数比Numpy函数快。因此，对于相同的操作，OpenCV函数是首选。但是，也有例外，尤其是当Numpy使用视图而不是副本时。

<h3>8.4 更多的 IPython 黑魔法</h3>

还有其他一些神奇的命令可以用来度量性能、分析、行分析、内存测量等等，它们都有很好的文档记录。所以这里只提供给这些文档的链接。有兴趣的读者可以尝试一下。

<h3>8.5 效率优化技术</h3>



有几种技术和编码方法可以利用Python和Numpy的最大性能。这里只注意到相关的信息，并链接到重要的来源。这里需要注意的是，首先尝试以一种简单的方式实现该算法。一旦它开始工作，剖析它，找出瓶颈并优化它们。

<ol><li>尽量避免在Python中使用循环，特别是双/三重循环等。</li>

<li>将算法/代码量化到最大可能的程度，因为Numpy和OpenCV对向量操作进行了优化。</li>

<li>利用缓存一致性。</li>

<li>除非需要，否则不要复制数组。尽量使用视图。数组复制是一项开销很大的操作。</li>
</ol>

即使在完成所有这些操作之后，如果代码仍然很慢，或者使用大的循环是不可避免的，那么使用像Cython这样的额外的库使它更快。